### matrix factorization 추천 시스템

# 협력 필터링

In [1]:
import numpy as np
import pandas as pd
import math
import scipy as sp
import scipy.sparse.linalg as splinalg

C:\Users\kt826\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv("C:/Users/kt826/Desktop/kmeansclustering1.csv")
data

,food_name,kal,0,1,2,3,4,5,6,7,...,176,177,178,179,180,181,182,183,184,185
0,감자미음,40.30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,고구마미음,47.60,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,브로콜리미음,39.00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,사과미음,47.46,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,쌀미음,52.20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,사과치즈진밥,168.30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144,명태당근진밥,140.60,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
145,감자사과메시,57.15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146,달걀노른자죽,137.40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
# 관련 없는 열 제외하기
baby_food_matrix = data.iloc[:,2:]

# 행과 열 바꾸기
baby_food_matrix = baby_food_matrix.T

# 열의 인덱스의 이름을 'food_id'로 변경
baby_food_matrix.columns.name = 'food_id'

# 행의 인덱스의 이름을 'ingredient_id'로 변경
baby_food_matrix.index.name = 'ingredient_id'

baby_food_matrix

food_id,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
ingredient_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
182,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
baby_food_matrix.dtypes

food_id
0      int64
1      int64
2      int64
3      int64
4      int64
       ...  
143    int64
144    int64
145    int64
146    int64
147    int64
Length: 148, dtype: object

In [5]:
baby_food_matrix = baby_food_matrix.astype(np.float32)

In [6]:
# SVD 코드 구현
def get_svd_prediction(baby_food_matrix,k):
    # U, sigma, V를 얻어오기
    u,s,vh = sp.sparse.linalg.svds(baby_food_matrix.to_numpy(),k=k)
    # 이들을 다시 곱해서 원본 행렬 복원
    # sigma는 주대각선 성분만 반환되므로 np.diag 함수로 대각행렬로 바꿔줌
    preds = np.dot(np.dot(u, np.diag(s)),vh)
    
    # 결과를 Dataframe으로 만들고, 0-1 사이의 값으로 정규화
    preds = np.dot(np.dot(u,np.diag(s)),vh)
    preds = pd.DataFrame(preds, columns=baby_food_matrix.columns, index=baby_food_matrix.index)
    preds = (preds-preds.min()) / (preds.max()-preds.min())
    return preds

predictions = get_svd_prediction(baby_food_matrix, k=10)
result = pd.DataFrame(data=predictions)
result

food_id,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
ingredient_id,,,,,,,,,,,,,,,,,,,,,
0,0.182534,0.208874,0.142286,0.162500,0.136489,0.264829,0.108884,0.134852,0.216632,0.195743,...,0.279786,0.276514,0.131796,0.231598,0.125578,0.183204,0.199218,0.092882,0.149574,0.318338
1,0.178419,0.206544,0.141655,0.156875,0.122898,0.205122,0.058571,0.129760,0.224756,0.197207,...,0.119931,0.225232,0.155648,0.231642,0.190249,0.166584,0.242066,0.124949,0.087245,0.154052
2,0.187593,0.216921,0.121517,0.164018,0.117401,0.230425,0.070544,0.124810,0.233254,0.193129,...,0.109764,0.174917,0.155988,0.175477,0.168609,0.131011,0.127146,0.125333,0.110823,0.118411
3,0.183865,0.221131,0.134206,0.170408,0.119684,0.225560,0.069074,0.130109,0.232977,0.198248,...,0.105080,0.171377,0.159451,0.172564,0.170273,0.133917,0.123895,0.121931,0.120761,0.119620
4,0.239770,0.273674,0.197959,0.210630,0.173691,0.278485,0.064168,0.190153,0.295170,0.258938,...,0.116256,0.190254,0.189378,0.216992,0.187442,0.165035,0.142438,0.118872,0.105521,0.138537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0.224342,0.350699,0.270113,0.146098,0.063263,0.257361,0.243056,0.134912,0.196202,0.179362,...,0.109711,0.172381,0.253074,0.000000,0.108716,0.000000,0.126257,0.125671,0.692098,0.116162
182,0.253842,0.275758,0.159461,0.165874,0.153432,0.258044,0.067054,0.165107,0.297388,0.234174,...,0.109789,0.149154,0.189672,0.176716,0.165986,0.101439,0.114376,0.106547,0.095926,0.080804
183,0.175941,0.199186,0.103280,0.151185,0.098530,0.210792,0.063492,0.105079,0.215234,0.173754,...,0.082578,0.173275,0.150023,0.174113,0.181147,0.127252,0.153547,0.147279,0.100208,0.105344


In [7]:
result['food_id'] = range(len(result))
result

food_id,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,food_id
ingredient_id,,,,,,,,,,,,,,,,,,,,,
0,0.182534,0.208874,0.142286,0.162500,0.136489,0.264829,0.108884,0.134852,0.216632,0.195743,...,0.276514,0.131796,0.231598,0.125578,0.183204,0.199218,0.092882,0.149574,0.318338,0
1,0.178419,0.206544,0.141655,0.156875,0.122898,0.205122,0.058571,0.129760,0.224756,0.197207,...,0.225232,0.155648,0.231642,0.190249,0.166584,0.242066,0.124949,0.087245,0.154052,1
2,0.187593,0.216921,0.121517,0.164018,0.117401,0.230425,0.070544,0.124810,0.233254,0.193129,...,0.174917,0.155988,0.175477,0.168609,0.131011,0.127146,0.125333,0.110823,0.118411,2
3,0.183865,0.221131,0.134206,0.170408,0.119684,0.225560,0.069074,0.130109,0.232977,0.198248,...,0.171377,0.159451,0.172564,0.170273,0.133917,0.123895,0.121931,0.120761,0.119620,3
4,0.239770,0.273674,0.197959,0.210630,0.173691,0.278485,0.064168,0.190153,0.295170,0.258938,...,0.190254,0.189378,0.216992,0.187442,0.165035,0.142438,0.118872,0.105521,0.138537,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0.224342,0.350699,0.270113,0.146098,0.063263,0.257361,0.243056,0.134912,0.196202,0.179362,...,0.172381,0.253074,0.000000,0.108716,0.000000,0.126257,0.125671,0.692098,0.116162,181
182,0.253842,0.275758,0.159461,0.165874,0.153432,0.258044,0.067054,0.165107,0.297388,0.234174,...,0.149154,0.189672,0.176716,0.165986,0.101439,0.114376,0.106547,0.095926,0.080804,182
183,0.175941,0.199186,0.103280,0.151185,0.098530,0.210792,0.063492,0.105079,0.215234,0.173754,...,0.173275,0.150023,0.174113,0.181147,0.127252,0.153547,0.147279,0.100208,0.105344,183


In [8]:
df = pd.DataFrame(baby_food_matrix)

ingredient_id = 147
# 해당 재료가 들어간 음식 추출
foods_with_ingredient = df[df[ingredient_id] == 1].index

fwi = pd.DataFrame([foods_with_ingredient])
fwi['ingredient_id'] = ingredient_id
fwi.set_index('ingredient_id', inplace=True)
fwi


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
ingredient_id,,,,,,,,,,,,,,
147,0,42,50,53,72,73,74,75,100,104,108,124,132,145


In [9]:
# 알고리즘화 for ingredient_id in range(0,185):

In [10]:
# 다음 음식 추천

# 복원된 행렬에서 재료 row만 가져온 뒤 내림차순으로 정렬
ingredient_predictions = result.loc[[str(ingredient_id)]].T.sort_values(by=str(ingredient_id), ascending=False)
# 수정
ingredient_predictions = ingredient_predictions.T
ingredient_predictions = ingredient_predictions.drop('food_id',axis=1)
ingredient_predictions

# 이전 음식 제외
ingredient_predictions = ingredient_predictions[~ingredient_predictions.index.isin(foods_with_ingredient)]

# 가장 예측값이 큰 10개 음식을 가져옵니다
ingredient_predictions = ingredient_predictions.head(10)
ingredient_predictions


food_id,116,120,81,65,74,64,112,124,97,70,...,53,62,131,122,137,132,35,42,6,84
ingredient_id,,,,,,,,,,,,,,,,,,,,,
147,0.365216,0.363363,0.325293,0.272344,0.267931,0.247929,0.239861,0.239173,0.237489,0.232769,...,0.085031,0.079432,0.077809,0.072389,0.071666,0.071343,0.067127,0.062066,0.061083,0.053381
